In [65]:
import pandas as pd
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors

In [33]:
df_gastos = pd.read_csv("../datos/gastos.csv")
df_gastos

,año,ncodi,totalcompra,producfarma,materialsani,implantes,restomateriasani,servcontratado,trabajocontratado,xrestocompras,variaexistencias,servexteriores,sumistro,xrestoserviexter,gastopersonal,sueldos,indemnizacion,segsocempresa,otrgassocial,dotaamortizacion,perdidadeterioro,xrestogasto,totcompragasto
0,2020,113.0,1535894,344330.0,169443,0.0,169443.0,1877.0,678201.0,342043.0,3039.0,719835,213817.0,506018.0,26209836,19875799,349499.0,5984538,0.0,298707.0,-6504.0,50165.0,28810972
1,2020,619.0,7884157,971762.0,1749006,493444.0,1255562.0,3800207.0,1248674.0,114508.0,24083.0,2844980,209305.0,2635675.0,7083969,5203283,151380.0,1591074,138232.0,2423752.0,0.0,312984.0,20573925
2,2020,163.0,5453290,126493.0,1412366,574127.0,838239.0,3217684.0,647852.0,48895.0,-26827.0,979733,97459.0,882274.0,2890388,2180177,-402.0,663398,47215.0,202688.0,-54028.0,232471.0,9677715
3,2020,131.0,245802,72318.0,52506,0.0,52506.0,1114.0,24303.0,95561.0,0.0,566931,79878.0,487053.0,1887375,1437421,9506.0,416398,24050.0,12084.0,240.0,132044.0,2844476
4,2020,375.0,762218,114702.0,106844,60.0,106784.0,14526.0,188463.0,337683.0,242.0,231222,39090.0,192132.0,7764302,5838470,0.0,1837175,88657.0,184413.0,93708.0,1552.0,9037657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8445,2012,970.0,271324,8510.0,166422,0.0,166422.0,0.0,96392.0,0.0,0.0,495914,405162.0,90752.0,1010376,796935,0.0,213441,0.0,104331.0,0.0,63796.0,1945741
8446,2012,1019.0,1687968,126665.0,94290,0.0,94290.0,1080335.0,300411.0,86267.0,-3612.0,228938,174167.0,54771.0,2447989,1852771,0.0,590887,4331.0,132364.0,0.0,0.0,4493647
8447,2012,1016.0,5418532,336983.0,1152639,145404.0,1007235.0,2739305.0,1126019.0,63586.0,0.0,1791212,430543.0,1360669.0,2624854,2066623,0.0,545427,12804.0,32691.0,99382.0,199998.0,10166669
8448,2012,1047.0,15537164,5516672.0,5255744,565150.0,4690594.0,0.0,3287539.0,1477209.0,-320553.0,7269413,2460724.0,4808689.0,43164526,34899381,0.0,8056716,208429.0,3297451.0,0.0,0.0,68948001


In [34]:
df_gastos.columns

Index(['año', 'ncodi', 'totalcompra', 'producfarma', 'materialsani',
       'implantes', 'restomateriasani', 'servcontratado', 'trabajocontratado',
       'xrestocompras', 'variaexistencias', 'servexteriores', 'sumistro',
       'xrestoserviexter', 'gastopersonal', 'sueldos', 'indemnizacion',
       'segsocempresa', 'otrgassocial', 'dotaamortizacion', 'perdidadeterioro',
       'xrestogasto', 'totcompragasto'],
      dtype='object')

In [38]:
df_ingresos = pd.read_csv("../datos/ingresos.csv", index_col=0)
df_ingresos

,particulares,aseguradoras,aseguradoras_enfermedad,aseguradoreas_trafico,mutuas,tipo,año,ncodi
0,0.0,103677.0,64383.0,39294.0,61456.0,Hospital,2013,"990,00"
1,324687.0,478280.0,392084.0,86196.0,484228.0,Hospital,2013,"680,00"
2,793512.0,1803352.0,1803352.0,0.0,20402.0,Hospital,2013,"411,00"
3,0.0,0.0,0.0,0.0,0.0,Hospital,2013,"241,00"
4,24406.0,63557.0,55091.0,8466.0,44106.0,Hospital,2013,"558,00"
...,...,...,...,...,...,...,...,...
55321,0.0,0.0,0.0,0.0,0.0,hospDom,2016,970
55322,0.0,0.0,0.0,0.0,0.0,hospDom,2016,1019
55323,0.0,0.0,0.0,0.0,0.0,hospDom,2016,1016
55324,0.0,0.0,0.0,0.0,0.0,hospDom,2016,1047


In [39]:
df_ingresos.dtypes

particulares               float64
aseguradoras               float64
aseguradoras_enfermedad    float64
aseguradoreas_trafico      float64
mutuas                     float64
tipo                        object
año                          int64
ncodi                       object
dtype: object

In [37]:
df_ingresos["tipo"].value_counts()

tipo
Hospital       9221
consulExter    9221
CMA            9221
hospDia        9221
Urgencia       9221
hospDom        9221
Name: count, dtype: int64

In [ ]:
df_ingresos[(df_ingresos["aseguradoreas_trafico"] !=0) & (df_ingresos["aseguradoreas_trafico"].notna())]

,particulares,aseguradoras,aseguradoras_enfermedad,aseguradoreas_trafico,mutuas,año,ncodi,tipo_hospitalizacion
38,41599.0,644836.0,625112.0,19724.0,0.0,2020,109.0,hospdia
80,58190.0,276.0,270.0,6.0,0.0,2020,663.0,hospdia
96,7004.0,275660.0,21811.0,253849.0,25277.0,2020,348.0,hospdia
98,3362.0,16350.0,0.0,16350.0,0.0,2020,761.0,hospdia
115,5049.0,31855.0,31444.0,411.0,4663.0,2020,770.0,hospdia
...,...,...,...,...,...,...,...,...
25321,16810.0,205789.0,31474.0,174315.0,69616.0,2010,369.0,hospital
25323,77723.0,272183.0,228923.0,43260.0,808.0,2010,713.0,hospital
25331,70130.0,2823208.0,1335505.0,1487703.0,674476.0,2010,765.0,hospital
25332,12118.0,14770.0,1938.0,12832.0,2584.0,2010,329.0,hospital


In [ ]:
df_gastos["ncodi"].nunique()

856

In [ ]:
df_ingresos["tipo_hospitalizacion"].value_counts()

tipo_hospitalizacion
hospdia     8450
hospdom     8450
hospital    8450
Name: count, dtype: int64

In [ ]:
df_final.dtypes

particulares               float64
aseguradoras               float64
aseguradoras_enfermedad    float64
aseguradoreas_trafico      float64
mutuas                     float64
año                          int64
ncodi                      float64
tipo_hospitalizacion        object
totalcompra                  int64
producfarma                float64
materialsani                 int64
implantes                  float64
restomateriasani           float64
servcontratado             float64
trabajocontratado          float64
xrestocompras              float64
variaexistencias           float64
servexteriores               int64
sumistro                   float64
xrestoserviexter           float64
gastopersonal                int64
sueldos                      int64
indemnizacion              float64
segsocempresa                int64
otrgassocial               float64
dotaamortizacion           float64
perdidadeterioro           float64
xrestogasto                float64
totcompragasto      

In [46]:
df_ingresos["ncodi"] = df_ingresos["ncodi"].str.split(",", expand=True)[0]

In [51]:
df_ingresos["ncodi"] = df_ingresos["ncodi"].str.split(".", expand=True)[0]

In [ ]:
def transfor_to_int(ncodi):
    try:
        ncodi_int = int(ncodi)
        return ncodi_int
    except:
        return ncodi


In [61]:
df_ingresos["ncodi"] = df_ingresos["ncodi"].apply(transfor_to_int)

990
680
411
241
558
167
248
224
591
586
453
113
619
163
131
375
1090
764
14
657
362
989
145
511
530
6
161
623
489
875
940
946
1023
1054
1057
1031
1032
1064
226
687
745
109
808
971
128
919
725
3
386
306
15
264
867
910
1053
1055
1081
1099
529
345
515
174
523
697
239
287
377
692
335
435
423
351
810
830
892
921
991
992
993
1070
1095
1094
473
663
552
772
568
398
394
614
235
616
267
565
220
278
107
141
711
204
579
348
766
761
728
280
222
276
11
302
682
279
282
281
183
610
116
115
114
126
119
770
583
136
469
642
674
193
526
698
630
124
364
604
123
310
571
296
566
784
102
379
639
457
609
101
454
318
600
545
100
702
778
341
99
767
359
98
132
97
103
110
94
91
664
918
92
95
96
301
492
811
812
832
841
856
859
861
880
883
884
877
911
913
922
923
924
947
951
1103
972
973
974
975
960
1028
976
977
1020
994
995
1060
1005
1061
1006
1088
1089
1076
1078
1033
1036
1037
1039
1067
1065
1066
207
431
387
179
606
925
541
797
627
173
636
205
814
881
771
533
447
815
801
800
389
695
816
833
870
927
961
962
1079
11

In [64]:
df_ingresos["ncodi"].isna().sum()

np.int64(18)

In [68]:
try:  
    conexion = psycopg2.connect(
        database = "Hospital",
        user = "postgres",
        password = "admin",
        host = "localhost",
        port = "5432")
    
except OperationalError as e:
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print("La contraseña es erronea")
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexion")
    else:
        print(f"Ocurrió el error {e}")

cursor = conexion.cursor()

In [72]:
# Creación tabla ingresos
query_creacion_ingresos = """create table if not exists ingresos (
                        id_ingresos SERIAL primary key,
                        id_ncodi int not null,
                        anio int not null,
                        id_tipo_hospi int not null,
                        particulares float,
                        aseguradoras float,
                        aseguradoras_enfermedad float,
                        aseguradoras_trafico float,
                        mutuas float,
                        foreign key (id_ncodi) references ncodi(id_ncodi) on delete restrict on update cascade,
                        foreign key (id_tipo_hospi) references tipo_hospi(id_tipo_hospi) on delete restrict on update cascade
                        );"""

cursor.execute(query_creacion_ingresos)
conexion.commit()



In [ ]:
# Creación tabla ncodi
query_creacion_ncodi = """create table if not exists ncodi (
                        id_ncodi int primary key,
                        ncodi int not null
                        );"""

cursor.execute(query_creacion_ncodi)
conexion.commit()

# Creación tabla TipoHospi
query_creacion_tipo_hospi = """create table if not exists tipo_hospi (
                        id_tipo_hospi int primary key,
                        nombre varchar(100) not null
                        );"""

cursor.execute(query_creacion_tipo_hospi)
conexion.commit()


# Creación tabla gastos
query_creacion_gastos = """create table if not exists gastos (
                        id_gastos SERIAL primary key,
                        id_ncodi int not null,
                        anio int not null,
                        totalcompra int,
                        producto_farma float,
                        material_sanitario int,
                        implantes float,
                        resto_material_sanitario float,
                        servicio_contratado float,
                        trabajo_contratado float,
                        resto_compras float,
                        varias_existencias float,
                        servicios_exteriores int,
                        suministro float,
                        resto_servicios_exteriores float,
                        gasto_personal int,
                        sueldos int,
                        indemnizacion float,
                        ss_empresa int,
                        ss_otra float,
                        dotacion_amortizacion float,
                        perdida_deterioro float,
                        resto_gastos float,
                        total_compra_gastos int,
                        foreign key (id_ncodi) references ncodi(id_ncodi) on delete restrict on update cascade
                        );"""

cursor.execute(query_creacion_gastos)
conexion.commit()


# Creación tabla ingresos
query_creacion_ingresos = """create table if not exists ingresos (
                        id_ingresos SERIAL primary key,
                        id_ncodi int not null,
                        anio int not null,
                        id_tipo_hospi int not null,
                        particulares float,
                        aseguradoras float,
                        aseguradoras_enfermedad float,
                        aseguradoras_trafico float,
                        mutuas float,
                        foreign key (id_ncodi) references ncodi(id_ncodi) on delete restrict on update cascade,
                        foreign key (id_tipo_hospi) references tipo_hospi(id_tipo_hospi) on delete restrict on update cascade
                        );"""

cursor.execute(query_creacion_ingresos)
conexion.commit()

